<p>Convolutional Neural Networks are very similar to ordinary Neural Networks from the previous chapter: they are made up of neurons that have learnable weights and biases. Each neuron receives some inputs, performs a dot product and optionally follows it with a non-linearity. The whole network still expresses a single differentiable score function: from the raw image pixels on one end to class scores at the other. And they still have a loss function (e.g. SVM/Softmax) on the last (fully-connected) layer and all the tips/tricks we developed for learning regular Neural Networks still apply.</p>

<p>So what changes? ConvNet architectures make the explicit assumption that the inputs are images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.</p>

<p><a name="overview"></a></p>

<h1 id="architecture-overview">1. Architecture Overview</h1>

<p><em>Recall: Regular Neural Nets.</em> As we saw in the previous chapter, Neural Networks receive an input (a single vector), and transform it through a series of <em>hidden layers</em>. Each hidden layer is made up of a set of neurons, where each neuron is fully connected to all neurons in the previous layer, and where neurons in a single layer function completely independently and do not share any connections. The last fully-connected layer is called the “output layer” and in classification settings it represents the class scores.</p>

<p><em>Regular Neural Nets don’t scale well to full images</em>. In CIFAR-10, images are only of size 32x32x3 (32 wide, 32 high, 3 color channels), so a single fully-connected neuron in a first hidden layer of a regular Neural Network would have 32*32*3 = 3072 weights. This amount still seems manageable, but clearly this fully-connected structure does not scale to larger images. For example, an image of more respectable size, e.g. 200x200x3, would lead to neurons that have 200*200*3 = 120,000 weights. Moreover, we would almost certainly want to have several such neurons, so the parameters would zadd up quickly! Clearly, this full connectivity is wasteful and the huge number of parameters would quickly lead to overfitting.</p>

<p><em>3D volumes of neurons</em>. Convolutional Neural Networks take advantage of the fact that the input consists of images and they constrain the architecture in a more sensible way. In particular, unlike a regular Neural Network, the layers of a ConvNet have neurons arranged in 3 dimensions: <strong>width, height, depth</strong>. (Note that the word <em>depth</em> here refers to the third dimension of an activation volume, not to the depth of a full Neural Network, which can refer to the total number of layers in a network.) For example, the input images in CIFAR-10 are an input volume of activations, and the volume has dimensions 32x32x3 (width, height, depth respectively). As we will soon see, the neurons in a layer will only be connected to a small region of the layer before it, instead of all of the neurons in a fully-connected manner. Moreover, the final output layer would for CIFAR-10 have dimensions 1x1x10, because by the end of the ConvNet architecture we will reduce the full image into a single vector of class scores, arranged along the depth dimension. Here is a visualization:</p>


<div style="content: "";clear: both;display: table;">
  <div style="float: left;width: 50%;padding: 10px;">
    <img src="figures/neural_net2.jpeg" alt="Snow" style="width:70%">
  </div>
  <div style="float: left;width: 50%;padding: 10px;">
    <img src="figures/cnn.jpeg" alt="Forest" style="width:100%; border-left: 1px solid black;">
  </div>
</div>


<br>
<br><br><br>
Left: A regular 3-layer Neural Network. Right: A ConvNet arranges its neurons in three dimensions (width, height, depth), as visualized in one of the layers. Every layer of a ConvNet transforms the 3D input volume to a 3D output volume of neuron activations. In this example, the red input layer holds the image, so its width and height would be the dimensions of the image, and the depth would be 3 (Red, Green, Blue channels).

<blockquote>
  <p>A ConvNet is made up of Layers. Every Layer has a simple API: It transforms an input 3D volume to an output 3D volume with some differentiable function that may or may not have parameters.</p>
</blockquote>

<p><a name="layers"></a></p>

<h1 id="layers-used-to-build-convnets">2. Layers used to build ConvNets</h1>

<p>As we described above, a simple ConvNet is a sequence of layers, and every layer of a ConvNet transforms one volume of activations to another through a differentiable function. We use three main types of layers to build ConvNet architectures: <strong>Convolutional Layer</strong>, <strong>Pooling Layer</strong>, and <strong>Fully-Connected Layer</strong> (exactly as seen in regular Neural Networks). We will stack these layers to form a full ConvNet <strong>architecture</strong>.</p>

<p><em>Example Architecture: Overview</em>. We will go into more details below, but a simple ConvNet for CIFAR-10 classification could have the architecture [INPUT - CONV - RELU - POOL - FC]. In more detail:</p>

> <ul>
  <li>INPUT [32x32x3] will hold the raw pixel values of the image, in this case an image of width 32, height 32, and with three color channels R,G,B.</li>
  <li>CONV layer will compute the output of neurons that are connected to local regions in the input, each computing a dot product between their weights and a small region they are connected to in the input volume. This may result in volume such as [32x32x12] if we decided to use 12 filters.</li>
  <li>RELU layer will apply an elementwise activation function, such as the \(max(0,x)\) thresholding at zero. This leaves the size of the volume unchanged ([32x32x12]).</li>
  <li>POOL layer will perform a downsampling operation along the spatial dimensions (width, height), resulting in volume such as [16x16x12].</li>
  <li>FC (i.e. fully-connected) layer will compute the class scores, resulting in volume of size [1x1x10], where each of the 10 numbers correspond to a class score, such as among the 10 categories of CIFAR-10. As with ordinary Neural Networks and as the name implies, each neuron in this layer will be connected to all the numbers in the previous volume.</li>
</ul>

<p>In this way, ConvNets transform the original image layer by layer from the original pixel values to the final class scores. Note that some layers contain parameters and other don’t. In particular, the CONV/FC layers perform transformations that are a function of not only the activations in the input volume, but also of the parameters (the weights and biases of the neurons). On the other hand, the RELU/POOL layers will implement a fixed function. The parameters in the CONV/FC layers will be trained with gradient descent so that the class scores that the ConvNet computes are consistent with the labels in the training set for each image.</p>

<p>In summary:</p>

<ul>
  <li>A ConvNet architecture is in the simplest case a list of Layers that transform the image volume into an output volume (e.g. holding the class scores)</li>
  <li>There are a few distinct types of Layers (e.g. CONV/FC/RELU/POOL are by far the most popular)</li>
  <li>Each Layer accepts an input 3D volume and transforms it to an output 3D volume through a differentiable function</li>
  <li>Each Layer may or may not have parameters (e.g. CONV/FC do, RELU/POOL don’t)</li>
  <li>Each Layer may or may not have additional hyperparameters (e.g. CONV/FC/POOL do, RELU doesn’t)</li>
</ul>

<div class="fig figcenter fighighlight">
  <img src="figures/convnet.jpeg" width="80%" style="padding:10px"/>
</div>    

The activations of an example ConvNet architecture. The initial volume stores the raw image pixels (left) and the last volume stores the class scores (right). Each volume of activations along the processing path is shown as a column. Since it's difficult to visualize 3D volumes, we lay out each volume's slices in rows. The last layer volume holds the scores for each class, but here we only visualize the sorted top 5 scores, and print the labels of each one. 

<p>We now describe the individual layers and the details of their hyperparameters and their connectivities.</p>

<p><a name="conv"></a></p>

<h2 id="convolutional-layer">2.1. Convolutional Layer</h2>

<p>The Conv layer is the core building block of a Convolutional Network that does most of the computational heavy lifting.</p>

<p><strong>Overview and intuition without brain stuff.</strong> Let’s first discuss what the CONV layer computes without brain/neuron analogies. The CONV layer’s parameters consist of a set of learnable filters. Every filter is small spatially (along width and height), but extends through the full depth of the input volume. For example, a typical filter on a first layer of a ConvNet might have size 5x5x3 (i.e. 5 pixels width and height, and 3 because images have depth 3, the color channels). During the forward pass, we slide (more precisely, convolve) each filter across the width and height of the input volume and compute dot products between the entries of the filter and the input at any position. As we slide the filter over the width and height of the input volume we will produce a 2-dimensional activation map that gives the responses of that filter at every spatial position. Intuitively, the network will learn filters that activate when they see some type of visual feature such as an edge of some orientation or a blotch of some color on the first layer, or eventually entire honeycomb or wheel-like patterns on higher layers of the network. Now, we will have an entire set of filters in each CONV layer (e.g. 12 filters), and each of them will produce a separate 2-dimensional activation map. We will stack these activation maps along the depth dimension and produce the output volume.</p>

<img src="figures/conv_layer.gif">

There are four hyperparameters that control the convolutional layer.

* Spatial extent: also called the filter size.
* Stride
* Padding
* Number of filters


<h3> Spatial extent </h3>

When dealing with high-dimensional inputs such as images, as we saw above it is impractical to connect neurons to all neurons in the previous volume. Instead, we will connect each neuron to only a local region of the input volume. <p> This is called <strong>Local Connectivity.</strong>. The spatial extent of this connectivity is a hyperparameter called the <strong>receptive field</strong> of the neuron (equivalently this is the filter size). The extent of the connectivity along the depth axis is always equal to the depth of the input volume. It is important to emphasize again this asymmetry in how we treat the spatial dimensions (width and height) and the depth dimension: The connections are local in space (along width and height), but always full along the entire depth of the input volume.</p>

<img src="figures/convolution_on_volume.png" width="60%">

> <p><em>Example 1</em>. For example, suppose that the input volume has size [32x32x3], (e.g. an RGB CIFAR-10 image). If the spatial extent (or the filter size) is 5x5, then each neuron in the Conv Layer will have weights to a [5x5x3] region in the input volume, for a total of 5*5*3 = 75 weights (and +1 bias parameter). Notice that the extent of the connectivity along the depth axis must be 3, since this is the depth of the input volume.</p>


> <p><em>Example 2</em>. Suppose an input volume had size [16x16x20]. Then using an example kernel size of 3x3, every neuron in the Conv Layer would now have a total of 3*3*20 = 180 connections to the input volume. Notice that, again, the connectivity is local in space (e.g. 3x3), but full along the input depth (20).</p>

<h3> Stride </h3>

Stride governs how many cells the filter is moved in the input to calculate the next cell in the result.

<img src="figures/stride.png" width="60%">

<h3> Padding </h3>

Padding has the following benefits:

* It allows us to use a CONV layer without necessarily shrinking the height and width of the volumes. This is important for building deeper networks, since otherwise the height/width would shrink as we go to deeper layers.
* It helps us keep more of the information at the border of an image. Without padding, very few values at the next layer would be affected by pixels as the edges of an image.

<img src="figures/padding.png" width="60%">

Notice the the dimension of the result has changed due to padding. 

Some padding terminologies:

* “valid” padding: no padding
* “same” padding: padding so that the output dimension is the same as the input


<h3> Multiple Filters </h3>


Multiple filters can be used in a convolution layer to detect multiple features. The output of the layer then will have the same number of channels as the number of filters in the layer.

<img src="figures/multiple_filters.png" width="60%">    
    

<h3> Spatial arrangement </h3>

<p>We have explained the connectivity of each neuron in the Conv Layer to the input volume, but we haven’t yet discussed how many neurons there are in the output volume or how they are arranged. 

<p>We can compute the spatial size of the output volume as a function of the input volume size (\(W\)), the filter size of the Conv Layer neurons (\(F\)), the stride with which they are applied (\(S\)), and the amount of zero padding used (\(P\)) on the border. You can convince yourself that the correct formula for calculating how many neurons “fit” is given by \((W - F + 2P)/S + 1\). For example for a 7x7 input and a 3x3 filter with stride 1 and pad 0 we would get a 5x5 output. With stride 2 we would get a 3x3 output.</p>

<p><strong>Summary</strong>. To summarize, the Conv Layer:</p>

<ul>
  <li>Accepts a volume of size \(W_1 \times H_1 \times D_1\)</li>
  <li>Requires four hyperparameters:
    <ul>
      <li>Number of filters \(K\),</li>
      <li>their spatial extent \(F\),</li>
      <li>the stride \(S\),</li>
      <li>the amount of zero padding \(P\).</li>
    </ul>
  </li>
  <li>Produces a volume of size \(W_2 \times H_2 \times D_2\) where:
    <ul>
      <li>\(W_2 = (W_1 - F + 2P)/S + 1\)</li>
      <li>\(H_2 = (H_1 - F + 2P)/S + 1\) (i.e. width and height are computed equally by symmetry)</li>
      <li>\(D_2 = K\)</li>
    </ul>
  </li>
  <li>With parameter sharing, it introduces \(F \cdot F \cdot D_1\) weights per filter, for a total of \((F \cdot F \cdot D_1) \cdot K\) weights and \(K\) biases.</li>
  <li>In the output volume, the \(d\)-th depth slice (of size \(W_2 \times H_2\)) is the result of performing a valid convolution of the \(d\)-th filter over the input volume with a stride of \(S\), and then offset by \(d\)-th bias.</li>
</ul>

<p>A common setting of the hyperparameters is \(F = 3, S = 1, P = 1\). However, there are common conventions and rules of thumb that motivate these hyperparameters. See the <a href="#architectures">ConvNet architectures</a> section below.</p>

<h3>Convolution Demo</h3>. 
    
<p>Below is a running demo of a CONV layer. Since 3D volumes are hard to visualize, all the volumes (the input volume (in blue), the weight volumes (in red), the output volume (in green)) are visualized with each depth slice stacked in rows. The input volume is of size \(W_1 = 5, H_1 = 5, D_1 = 3\), and the CONV layer parameters are \(K = 2, F = 3, S = 2, P = 1\). That is, we have two filters of size \(3 \times 3\), and they are applied with a stride of 2. Therefore, the output volume size has spatial size (5 - 3 + 2)/2 + 1 = 3. Moreover, notice that a padding of \(P = 1\) is applied to the input volume, making the outer border of the input volume zero. The visualization below iterates over the output activations (green), and shows that each element is computed by elementwise multiplying the highlighted input (blue) with the filter (red), summing it up, and then offsetting the result by the bias.</p>

<div class="fig figcenter fighighlight">
  <img src="figures/conv_demo.gif" width="90%" height="700px;" style="border:none; padding: 10px">
  <div class="figcaption"></div>
</div>

<p><a name="pool"></a></p>
<h2 id="pooling-layer">2.2. Pooling Layer</h2>

<p>It is common to periodically insert a Pooling layer in-between successive Conv layers in a ConvNet architecture. Its function is to progressively reduce the spatial size of the representation to reduce the amount of parameters and computation in the network, and hence to also control overfitting. The Pooling Layer operates independently on every depth slice of the input and resizes it spatially, using the MAX operation. The most common form is a pooling layer with filters of size 2x2 applied with a stride of 2 downsamples every depth slice in the input by 2 along both width and height, discarding 75% of the activations. Every MAX operation would in this case be taking a max over 4 numbers (little 2x2 region in some depth slice). 
    
<img src="figures/maxpool.gif" width="500px" style="padding:10px"/>

When done on input with multiple channels, pooling reduces the height and width but keeps the depth dimension unchanged.

<img src="figures/pooling_on_volume.png" width="500px" style="padding:10px"/>

    
More generally, the pooling layer:</p>

<ul>
  <li>Accepts a volume of size \(W_1 \times H_1 \times D_1\)</li>
  <li>Requires two hyperparameters:
    <ul>
      <li>their spatial extent \(F\),</li>
      <li>the stride \(S\),</li>
    </ul>
  </li>
  <li>Produces a volume of size \(W_2 \times H_2 \times D_2\) where:
    <ul>
      <li>\(W_2 = (W_1 - F)/S + 1\)</li>
      <li>\(H_2 = (H_1 - F)/S + 1\)</li>
      <li>\(D_2 = D_1\)</li>
    </ul>
  </li>
  <li>Introduces zero parameters since it computes a fixed function of the input</li>
  <li>For Pooling layers, it is not common to pad the input using zero-padding.</li>
</ul>

<p>It is worth noting that there are only two commonly seen variations of the max pooling layer found in practice: A pooling layer with \(F = 3, S = 2\) (also called overlapping pooling), and more commonly \(F = 2, S = 2\). Pooling sizes with larger receptive fields are too destructive.</p>

<p><strong>General pooling</strong>. In addition to max pooling, the pooling units can also perform other functions, such as <em>average pooling</em> or even <em>L2-norm pooling</em>. Average pooling was often used historically but has recently fallen out of favor compared to the max pooling operation, which has been shown to work better in practice.</p>

<p><strong>Getting rid of pooling</strong>. Many people dislike the pooling operation and think that we can get away without it. For example, <a href="http://arxiv.org/abs/1412.6806">Striving for Simplicity: The All Convolutional Net</a> proposes to discard the pooling layer in favor of architecture that only consists of repeated CONV layers. To reduce the size of the representation they suggest using larger stride in CONV layer once in a while. Discarding pooling layers has also been found to be important in training good generative models, such as variational autoencoders (VAEs) or generative adversarial networks (GANs). It seems likely that future architectures will feature very few to no pooling layers.</p>

<p><a name="norm"></a></p>

<h2 id="normalization-layer">2.3. Normalization Layer</h2>

<p>Many types of normalization layers have been proposed for use in ConvNet architectures, sometimes with the intentions of implementing inhibition schemes observed in the biological brain. However, these layers have since fallen out of favor because in practice their contribution has been shown to be minimal, if any. For various types of normalizations, see the discussion in Alex Krizhevsky’s <a href="http://code.google.com/p/cuda-convnet/wiki/LayerParams#Local_response_normalization_layer_(same_map)">cuda-convnet library API</a>.</p>

<p><a name="fc"></a></p>

<h2 id="fully-connected-layer">2.4. Fully-connected layer</h2>

<p>Neurons in a fully connected layer have full connections to all activations in the previous layer, as seen in regular Neural Networks. Their activations can hence be computed with a matrix multiplication followed by a bias offset.
    
    
<img src="figures/convNet_dim.png" alt="convdim">
    

<p><a name="architectures"></a></p>

<h1 id="convnet-architectures">3. ConvNet Architectures</h1>

<p>We have seen that Convolutional Networks are commonly made up of only three layer types: CONV, POOL (we assume Max pool unless stated otherwise) and FC (short for fully-connected). We will also explicitly write the RELU activation function as a layer, which applies elementwise non-linearity. In this section we discuss how these are commonly stacked together to form entire ConvNets.</p>

<p><a name="layerpat"></a></p>

<h2 id="layer-patterns">3.1. Layer Patterns</h2>
<p>The most common form of a ConvNet architecture stacks a few CONV-RELU layers, follows them with POOL layers, and repeats this pattern until the image has been merged spatially to a small size. At some point, it is common to transition to fully-connected layers. The last fully-connected layer holds the output, such as the class scores. In other words, the most common ConvNet architecture follows the pattern:</p>

<p><code class="language-plaintext highlighter-rouge">INPUT -&gt; [[CONV -&gt; RELU]*N -&gt; POOL?]*M -&gt; [FC -&gt; RELU]*K -&gt; FC</code></p>

<p>where the <code class="language-plaintext highlighter-rouge">*</code> indicates repetition, and the <code class="language-plaintext highlighter-rouge">POOL?</code> indicates an optional pooling layer. Moreover, <code class="language-plaintext highlighter-rouge">N &gt;= 0</code> (and usually <code class="language-plaintext highlighter-rouge">N &lt;= 3</code>), <code class="language-plaintext highlighter-rouge">M &gt;= 0</code>, <code class="language-plaintext highlighter-rouge">K &gt;= 0</code> (and usually <code class="language-plaintext highlighter-rouge">K &lt; 3</code>). For example, here are some common ConvNet architectures you may see that follow this pattern:</p>

<ul>
  <li><code class="language-plaintext highlighter-rouge">INPUT -&gt; FC</code>, implements a linear classifier. Here <code class="language-plaintext highlighter-rouge">N = M = K = 0</code>.</li>
  <li><code class="language-plaintext highlighter-rouge">INPUT -&gt; CONV -&gt; RELU -&gt; FC</code></li>
  <li><code class="language-plaintext highlighter-rouge">INPUT -&gt; [CONV -&gt; RELU -&gt; POOL]*2 -&gt; FC -&gt; RELU -&gt; FC</code>. Here we see that there is a single CONV layer between every POOL layer.</li>
  <li><code class="language-plaintext highlighter-rouge">INPUT -&gt; [CONV -&gt; RELU -&gt; CONV -&gt; RELU -&gt; POOL]*3 -&gt; [FC -&gt; RELU]*2 -&gt; FC</code> Here we see two CONV layers stacked before every POOL layer. This is generally a good idea for larger and deeper networks, because multiple stacked CONV layers can develop more complex features of the input volume before the destructive pooling operation.</li>
</ul>

<p><em>Prefer a stack of small filter CONV to one large receptive field CONV layer</em>. Suppose that you stack three 3x3 CONV layers on top of each other (with non-linearities in between, of course). In this arrangement, each neuron on the first CONV layer has a 3x3 view of the input volume. A neuron on the second CONV layer has a 3x3 view of the first CONV layer, and hence by extension a 5x5 view of the input volume. Similarly, a neuron on the third CONV layer has a 3x3 view of the 2nd CONV layer, and hence a 7x7 view of the input volume. Suppose that instead of these three layers of 3x3 CONV, we only wanted to use a single CONV layer with 7x7 receptive fields. These neurons would have a receptive field size of the input volume that is identical in spatial extent (7x7), but with several disadvantages. First, the neurons would be computing a linear function over the input, while the three stacks of CONV layers contain non-linearities that make their features more expressive. Second, if we suppose that all the volumes have \(C\) channels, then it can be seen that the single 7x7 CONV layer would contain \(C \times (7 \times 7 \times C) = 49 C^2\) parameters, while the three 3x3 CONV layers would only contain \(3 \times (C \times (3 \times 3 \times C)) = 27 C^2\) parameters. Intuitively, stacking CONV layers with tiny filters as opposed to having one CONV layer with big filters allows us to express more powerful features of the input, and with fewer parameters. As a practical disadvantage, we might need more memory to hold all the intermediate CONV layer results if we plan to do backpropagation.</p>

<p><strong>Recent departures.</strong> It should be noted that the conventional paradigm of a linear list of layers has recently been challenged, in Google’s Inception architectures and also in current (state of the art) Residual Networks from Microsoft Research Asia. Both of these (see details below in case studies section) feature more intricate and different connectivity structures.</p>

<p><strong>In practice: use whatever works best on ImageNet</strong>. If you’re feeling a bit of a fatigue in thinking about the architectural decisions, you’ll be pleased to know that in 90% or more of applications you should not have to worry about these. I like to summarize this point as “<em>don’t be a hero</em>”: Instead of rolling your own architecture for a problem, you should look at whatever architecture currently works best on ImageNet, download a pretrained model and finetune it on your data. You should rarely ever have to train a ConvNet from scratch or design one from scratch.


<p><a name="layersizepat"></a></p>

<h2 id="layer-sizing-patterns">3.2. Layer Sizing Patterns</h2>

<p>Until now we’ve omitted mentions of common hyperparameters used in each of the layers in a ConvNet. We will first state the common rules of thumb for sizing the architectures and then follow the rules with a discussion of the notation:</p>

<p>The <strong>input layer</strong> (that contains the image) should be divisible by 2 many times. Common numbers include 32 (e.g. CIFAR-10), 64, 96 (e.g. STL-10), or 224 (e.g. common ImageNet ConvNets), 384, and 512.</p>

<p>The <strong>conv layers</strong> should be using small filters (e.g. 3x3 or at most 5x5), using a stride of \(S = 1\), and crucially, padding the input volume with zeros in such way that the conv layer does not alter the spatial dimensions of the input. That is, when \(F = 3\), then using \(P = 1\) will retain the original size of the input. When \(F = 5\), \(P = 2\). For a general \(F\), it can be seen that \(P = (F - 1) / 2\) preserves the input size. If you must use bigger filter sizes (such as 7x7 or so), it is only common to see this on the very first conv layer that is looking at the input image.</p>

<p>The <strong>pool layers</strong> are in charge of downsampling the spatial dimensions of the input. The most common setting is to use max-pooling with 2x2 receptive fields (i.e. \(F = 2\)), and with a stride of 2 (i.e. \(S = 2\)). Note that this discards exactly 75% of the activations in an input volume (due to downsampling by 2 in both width and height). Another slightly less common setting is to use 3x3 receptive fields with a stride of 2, but this makes. It is very uncommon to see receptive field sizes for max pooling that are larger than 3 because the pooling is then too lossy and aggressive. This usually leads to worse performance.</p>

<p><em>Reducing sizing headaches.</em> The scheme presented above is pleasing because all the CONV layers preserve the spatial size of their input, while the POOL layers alone are in charge of down-sampling the volumes spatially. In an alternative scheme where we use strides greater than 1 or don’t zero-pad the input in CONV layers, we would have to very carefully keep track of the input volumes throughout the CNN architecture and make sure that all strides and filters “work out”, and that the ConvNet architecture is nicely and symmetrically wired.</p>

<p><em>Why use stride of 1 in CONV?</em> Smaller strides work better in practice. Additionally, as already mentioned stride 1 allows us to leave all spatial down-sampling to the POOL layers, with the CONV layers only transforming the input volume depth-wise.</p>

<p><em>Why use padding?</em> In addition to the aforementioned benefit of keeping the spatial sizes constant after CONV, doing this actually improves performance. If the CONV layers were to not zero-pad the inputs and only perform valid convolutions, then the size of the volumes would reduce by a small amount after each CONV, and the information at the borders would be “washed away” too quickly.</p>

<p><em>Compromising based on memory constraints.</em> In some cases (especially early in the ConvNet architectures), the amount of memory can build up very quickly with the rules of thumb presented above. For example, filtering a 224x224x3 image with three 3x3 CONV layers with 64 filters each and padding 1 would create three activation volumes of size [224x224x64]. This amounts to a total of about 10 million activations, or 72MB of memory (per image, for both activations and gradients). Since GPUs are often bottlenecked by memory, it may be necessary to compromise. In practice, people prefer to make the compromise at only the first CONV layer of the network. For example, one compromise might be to use a first CONV layer with filter sizes of 7x7 and stride of 2 (as seen in a ZF net). As another example, an AlexNet uses filter sizes of 11x11 and stride of 4.</p>


<p><a name="case"></a></p>

<h2 id="case-studies">3.4. Case studies</h2>

We learned about Conv layer, pooling layer, and fully connected layers. It turns out that computer vision researchers spent the past few years on how to put these layers together. To get some intuitions you have to see the examples that has been made. Some neural networks architecture that works well in some tasks can also work well in other tasks

<p>There are several architectures in the field of Convolutional Networks that have a name. The most common are:</p>

<h3 id="LeNet">3.4.1. LeNet</h3>

The first successful applications of Convolutional Networks were developed by Yann LeCun in 1990’s. Of these, the best known is the <a href="http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf">LeNet</a> architecture that was used to read zip codes, digits, etc.
<ul>  
    <li> The goal for this model was to identify handwritten digits in a 32x32x1 gray image.
    <li> This model was published in 1998. The last layer wasn't using softmax back then.
    <li> It has 60k parameters. 
    <li> The dimensions of the image decreases as the number of channels increases. 
    <li> The activation function used in the paper was Sigmoid and Tanh. Modern implementation uses RELU in most of the cases. 
    <li> Number of parameters:  ~ 60 thousands.
</ul>
   
<img src="figures/leNet.png" alt="leNet" width="70%">

<h3 id="AlexNet">3.4.2. AlexNet</h3>

The first work that popularized Convolutional Networks in Computer Vision was the <a href="http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks">AlexNet</a>, developed by Alex Krizhevsky, Ilya Sutskever and Geoff Hinton. The AlexNet was submitted to the <a href="http://www.image-net.org/challenges/LSVRC/2014/">ImageNet ILSVRC challenge</a> in 2012 and significantly outperformed the second runner-up (top 5 error of 16% compared to runner-up with 26% error). The Network had a very similar architecture to LeNet, but was deeper, bigger, and featured Convolutional Layers stacked on top of each other (previously it was common to only have a single CONV layer always immediately followed by a POOL layer)
    
<ul>
    <li> Named after Alex Krizhevsky who was the first author of this paper. The other authors includes Geoffrey Hinton.
    <li> The goal for the model was the ImageNet challenge which classifies images into 1000 classes.
    <li> Similar to LeNet-5 but bigger.
    <li> Number of parameters:  ~ 60 millions.
    <li> It used the RELU activation function.
</ul>

<img src="figures/AlexNet.png" alt="AlexNet" width="80%"/>

<h3 id="ZF Net">3.4.3. ZF Net</h3>
    
The ILSVRC 2013 winner was a Convolutional Network from Matthew Zeiler and Rob Fergus. It became known as the <a href="http://arxiv.org/abs/1311.2901">ZFNet</a> (short for Zeiler &amp; Fergus Net). It was an improvement on AlexNet by tweaking the architecture hyperparameters, in particular by expanding the size of the middle convolutional layers and making the stride and filter size on the first layer smaller.

<h3 id="VGGNet">3.4.4. VGGNet</h3>

The runner-up in ILSVRC 2014 was the network from Karen Simonyan and Andrew Zisserman that became known as the <a href="http://www.robots.ox.ac.uk/~vgg/research/very_deep/">VGGNet</a>. Its main contribution was in showing that the depth of the network is a critical component for good performance. Their final best network contains 16 CONV/FC layers and, appealingly, features an extremely homogeneous architecture that only performs 3x3 convolutions and 2x2 pooling from the beginning to the end. A downside of the VGGNet is that it is more expensive to evaluate and uses a lot more memory and parameters (140M). Most of these parameters are in the first fully connected layer, and it was since found that these FC layers can be removed with no performance downgrade, significantly reducing the number of necessary parameters.

<ul>
    <li>  A modification for AlexNet.
    <li>  Instead of having a lot of hyperparameters lets have some simpler network.
    <li>  Focus on having only these blocks:
        <ul>
            <li> CONV = 3 X 3 filter, s = 1, "same" padding.
            <li> MAX-POOL = 2 X 2 , s = 2
        </ul>    
    <li>  This network is large even by modern standards. It has around 138 million parameters. Most of the parameters are in the fully connected layers.
    <li>  It has a total memory of 96MB per image for only forward propagation! Most memory are in the earlier layers.
    <li>  Number of filters increases from 64 to 128 to 256 to 512. 512 was made twice.
    <li>  Pooling was the only one who is responsible for shrinking the dimensions.
    <li>  There are another version called VGG-19 which is a bigger version. But most people uses the VGG-16 instead of the VGG-19 because it does the same.
</ul>

<img src="figures/vggNet.png" alt="vggNet" width="80%">

<h3 id="GoogLeNet">3.4.5. GoogLeNet (Inception)</h3>

The ILSVRC 2014 winner was a Convolutional Network from <a href="http://arxiv.org/abs/1409.4842">Szegedy et al.</a> from Google. Its main contribution was the development of an <em>Inception Module</em> that dramatically reduced the number of parameters in the network (4M, compared to AlexNet with 60M). Additionally, this paper uses Average Pooling instead of Fully Connected layers at the top of the ConvNet, eliminating a large amount of parameters that do not seem to matter much. There are also several followup versions to the GoogLeNet, most recently <a href="http://arxiv.org/abs/1602.07261">Inception-v4</a>.

<ul>
    <li>  <strong>Network in Network and 1 X 1 convolutions</strong>:
        
A 1 x 1 convolution, also called Network in Network, is so useful in many CNN models. It has been used in a lot of modern CNN implementations like ResNet and Inception models.

What does a 1 X 1 convolution do? Isn't it just multiplying by a number? A 1 x 1 convolution is useful when:
    <ul>
        <li> We want to shrink the number of channels. We also call this feature transformation. 
        <li> It can save a lot of computations.
        <li> If we have specified the number of 1 x 1 Conv filters to be the same as the input number of channels then the output will contain the same number of channels. Then the 1 x 1 Conv will act like a non linearity and will learn non linearity operator.
        <li> Some researchers replace fully connected layers with 1 x 1 convolutions as Yann LeCun believes they are the same.
    </ul>
    <li> <strong> Inception Module </strong>:

When you design a CNN you have to decide all the layers yourself. Will you pick a 3 x 3 Conv or 5 x 5 Conv or maybe a max pooling layer. You have so many choices. What inception tells us is, Why not use all of them at once?
        
<img src="figures/inception_module.png" alt="inception_module" width="40%">
       
The problem with the above network is computation cost (e.g. for the 5 x 5 filter only, the number of multiplications is (28 x 28 x 32) x (5 x 5 x 192) = ~ 120 millions). 
    <li> <strong> Inception Module with Dimension Reduction </strong>        
        
As mentioned above, inception modules face huge calculations. In order to reduce costs, the following figure shows the Inception module, authorAdded 1x1 convolution to limit the number of input channels before 3x3 and 5x5 convolution, that is, the dimension reduction is achieved. 

<img src="figures/inception_module_dimension_reduction.png" alt="inception_module_dimension_reduction" width="40%">

For the same 5 x 5 filter only, the number of multiplications now is:
    <ul>
        <li> For the first Conv: 28 * 28 * 16 * 1 * 1 * 192 = 2.5 Mil
        <li> For the second Conv: 28 * 28 * 32 * 5 * 5 * 16 = 10 Mil
        <li> So the total number are 12.5 Mil approx. which is so good compared to 120 Mil
    </ul>          
    <li> <strong> Inception Network </strong>   
        
Below is an inception network called GoogLeNet which has 9 inception modules. Some times a Max-Pool block is used before the inception module to reduce the dimensions of the inputs. There are 2 auxiliary classifiers at different positions in order to push the network toward its goal. This helps to ensure that the intermediate features are good enough to the network to learn and it turns out that softmax0 and sofmax1 gives regularization effect.
        
<img src="figures/GoogLeNet.png" alt="GoogLeNet" width="100%">                
</ul>        

        
        
<h3 id="ResNet">3.4.6. ResNet</h3>

<a href="http://arxiv.org/abs/1512.03385">Residual Network</a> developed by Kaiming He et al. was the winner of ILSVRC 2015. It features special <em>skip connections</em> and a heavy use of batch normalization. The architecture is also missing fully connected layers at the end of the network. ResNets are currently by far state of the art Convolutional Neural Network models and are the default choice for using ConvNets in practice (as of May 10, 2016). </li>
<ul>
    <li> Residual blocks — Building blocks of ResNet:
        <ul>
            <li> ResNets are built out of some Residual blocks.
            <li> In a residual block, they add a shortcut/skip connection before the second activation.
            <li> The activation at the second layer is hence $a^{[l+2]} = f(z^{[l+2]} + a^{[l]})$
            <li> The authors of this block find that you can train a deeper NNs by stacking this block.
        </ul>          
<img src="figures/residual_block.png" alt="residual_block" width="40%">
    <li> Residual Network:
        <ul>
            <li> A Residual Netwok is a NN that consists of some Residual blocks.                    
<img src="figures/resNet.png" alt="resNet">                    
            <li> These networks can go deeper without hurting the performance. In the normal NN - Plain networks - the theory tell us that if we go deeper we will get a better solution to our problem, but because of the vanishing and exploding gradients problems the performance of the network suffers as it goes deeper. Thanks to Residual Network we can go as deep as we want now.    
<img src="figures/plain_vs_resNet.png" alt="resNet" width="70%">
            <li> ResNets work good with deep layers and avoid the problem of vanishing activations. If $w^{[l+2]}$ has vanished, and $z^{[l+2]}$, then $a^{[l+2]} = f(z^{[l+2]} + a^{[l]}) = f(a^{[l]}) =  a^{[l]}$. This show that identity function is easy for a residual block to learn. And that why it can train deeper NNs.                    
        </ul>            
</ul>




